In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from ES import Normal, Expectation, params_to_model, model_to_params, normalize

def train(mu, sigma, npop, data_loader, model, optimizer, criterion, scheduler=None, device='cpu'):
    model.train()
    running_loss = 0
    
    normal = Normal(device)
    expectation = Expectation()
    
    prog_bar = tqdm(data_loader)
    for i, (source, target) in enumerate(prog_bar):
        source = source.to(device)
        target = target.to(device)

        theta, ratio = normal(mu, sigma, npop)
        score = torch.zeros_like(ratio)

        for j in range(npop):
            params_to_model(theta[j], model)
            output = model(source)
            score[j] = criterion(output, target).item()

        loss = expectation(normalize(score), ratio)
        running_loss += score.mean()
        prog_bar.set_description('Loss: {}'.format(running_loss/(i+1)))

        # Back-propagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler != None:
            scheduler.step()

    return running_loss / len(data_loader)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.num_filter1 = 8
        self.num_filter2 = 16
        self.num_padding = 2
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(1, self.num_filter1, 5, padding=self.num_padding)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(self.num_filter1, self.num_filter2, 5, padding=self.num_padding)
        # feature map size is 7*7 by pooling
        self.fc = nn.Linear(self.num_filter2*7*7, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_filter2*7*7)   # reshape Variable
        x = self.fc(x)
        return F.log_softmax(x, dim=-1)
    
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('.', train=True, download=True, transform = transform)
dataset2 = datasets.MNIST('.', train=False, download=True, transform = transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=128, shuffle=False)

model = Net().cuda()
for param in model.parameters():
    param.requires_grad = False
    
flat_params = torch.cat([param for param in model_to_params(model, flatten=True)])
mu = torch.tensor(flat_params, requires_grad=True, device='cuda')
sigma = torch.full(flat_params.shape, 0.01, requires_grad=True, device='cuda')
npop = 100
optimizer = torch.optim.AdamW([mu, sigma])
criterion = nn.NLLLoss()

for epoch in range(100): 
    loss = train(mu, sigma, npop, train_loader, model, optimizer, criterion, device=mu.device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


KeyboardInterrupt: ignored

In [31]:
# ------ Helper Functions ------
def data_to_device(data, device='cpu'):
	if isinstance(data, torch.Tensor):
		data = data.to(device)
	elif isinstance(data, tuple):
		data = tuple(data_to_device(item,device) for item in data)
	elif isinstance(data, list):
		data = list(data_to_device(item,device) for item in data)
	elif isinstance(data, dict):
		data = dict((k,data_to_device(v,device)) for k,v in data.items())
	else:
		raise TypeError('Unsupported Datatype! Must be a Tensor/List/Tuple/Dict.')
	return data

def data_concatenate(iterable_data, dim=0):
	data = iterable_data[0] # can be a list / tuple / dict / tensor
	if isinstance(data, torch.Tensor):
		return torch.cat([*iterable_data], dim=dim)
	elif isinstance(data, tuple):
		num_cols = len(data)
		num_rows = len(iterable_data)
		return_data = []
		for col in range(num_cols):
			data_col = []
			for row in range(num_rows):
				data_col.append(iterable_data[row][col])
			return_data.append(torch.cat([*data_col], dim=dim))
		return tuple(return_data)
	elif isinstance(data, list):
		num_cols = len(data)
		num_rows = len(iterable_data)
		return_data = []
		for col in range(num_cols):
			data_col = []
			for row in range(num_rows):
				data_col.append(iterable_data[row][col])
			return_data.append(torch.cat([*data_col], dim=dim))
		return list(return_data)
	elif isinstance(data, dict):
		num_cols = len(data)
		num_rows = len(iterable_data)
		return_data = []
		for col in data.keys():
			data_col = []
			for row in range(num_rows):
				data_col.append(iterable_data[row][col])
			return_data.append(torch.cat([*data_col], dim=dim))
		return dict((k,return_data[i]) for i,k in enumerate(data.keys()))
	else:
		raise TypeError('Unsupported Datatype! Must be a Tensor/List/Tuple/Dict.')

def data_distributor(model, source):
	if isinstance(source, torch.Tensor):
		output = model(source)
	elif isinstance(source, tuple) or isinstance(source, list):
		output = model(*source)
	elif isinstance(source, dict):
		output = model(**source)
	else:
		raise TypeError('Unsupported DataType! Try List/Tuple!')
	return output
	
def args_to_kwargs(args, kwargs_list=None): # This function helps distribute input to corresponding arguments in Torch models
	if kwargs_list != None:
		if isinstance(args, dict): # Nothing to do here
			return args 
		else: # args is a list or tuple or single element
			if isinstance(args, torch.Tensor): # single element
				args = [args]
			assert len(args) == len(kwargs_list)
			return dict(zip(kwargs_list, args))
	else: # Nothing to do here
		return args

# ------ Core Functions ------
# def train(data_loader, model, optimizer, criterion, scheduler=None, device='cpu', kw_src=None, kw_tgt=None, kw_out=None):
# 	model.train()
# 	running_loss = 0

# 	prog_bar = tqdm(data_loader)
# 	for i, (source, target) in enumerate(prog_bar):
# 		source = data_to_device(source, device)
# 		target = data_to_device(target, device)

# 		source = args_to_kwargs(source, kw_src)
# 		target = args_to_kwargs(target, kw_tgt)

# 		output = data_distributor(model, source)
# 		output = args_to_kwargs(output, kw_out)

# 		loss = criterion(output, target)
# 		running_loss += loss.item()
# 		prog_bar.set_description('Loss: {}'.format(running_loss/(i+1)))

# 		# Back-propagate and update weights
# 		optimizer.zero_grad()
# 		loss.backward()
# 		optimizer.step()
# 		if scheduler != None:
# 			scheduler.step()

# 	return running_loss / len(data_loader)

def test(data_loader, model, criterion=None, device='cpu', return_results=True, kw_src=None, kw_tgt=None, kw_out=None):
	model.eval()
	running_loss = 0

	outputs = []
	targets = []

	with torch.no_grad():
		prog_bar = tqdm(data_loader)
		for i, (source, target) in enumerate(prog_bar):
			source = data_to_device(source, device)
			target = data_to_device(target, device)

			source = args_to_kwargs(source, kw_src)
			target = args_to_kwargs(target, kw_tgt)

			output = data_distributor(model, source)
			output = args_to_kwargs(output, kw_out)

			if criterion != None:
				loss = criterion(output, target)
				running_loss += loss.item()
			prog_bar.set_description('Loss: {}'.format(running_loss/(i+1)))

			if return_results:
				outputs.append(data_to_device(output,'cpu'))
				targets.append(data_to_device(target,'cpu'))
	
	if return_results:
		outputs = data_concatenate(outputs)
		targets = data_concatenate(targets)
		return running_loss / len(data_loader), outputs, targets
	else:
		return running_loss / len(data_loader)

def save(path, model, optimizer=None, scheduler=None, epoch=-1, stats=None):
	torch.save({
		# --- Model Statistics ---
		'epoch': epoch,
		'stats': stats,
		# --- Model Parameters ---
		'model_state_dict': model.state_dict(),
		'optimizer_state_dict': optimizer.state_dict() if optimizer != None else None,
		'scheduler_state_dict': scheduler.state_dict() if scheduler != None else None,
	}, path)

def load(path, model, optimizer=None, scheduler=None):
	checkpoint = torch.load(path)
	# --- Model Statistics ---
	epoch = checkpoint['epoch']
	stats = checkpoint['stats']
	# --- Model Parameters ---
	model.load_state_dict(checkpoint['model_state_dict'])
	if optimizer != None:
		try:
			optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
		except: # Input optimizer doesn't fit the checkpoint one --> should be ignored
			print('Cannot load the optimizer')
	if scheduler != None:
		try:
			scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
		except: # Input scheduler doesn't fit the checkpoint one --> should be ignored
			print('Cannot load the scheduler')
	return epoch, stats

In [32]:
params_to_model(mu.detach(), model)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc): Linear(in_features=784, out_features=10, bias=True)
)

In [33]:
loss, test_outputs, test_targets = test(test_loader, model, device=mu.device)

In [34]:
import sklearn.metrics as metrics

In [35]:
metrics.accuracy_score(test_targets.cpu(), test_outputs.cpu().argmax(-1))

0.9387